In [31]:
import csv
import numpy as np

Setting

In [2]:
action = (0, 1)

In [3]:
def prob_gen(L):
    # set the edge case for state one 
    state_one = [[0, 0] for i in range(L)]
    state_one[0] = [1, 0.6]
    state_one[1] = [0, 0.4]
    # set the edge case for and stage L
    state_L = [[0, 0] for i in range(L)]
    state_L[-1] = [0, 0.6]
    state_L[-2] = [1, 0.4]
    # initialise the matrix
    states = [[] for i in range(L)]
    # general cases between state one and state L
    probs = [[1, 0.05], [0, 0.55], [0, 0.4]]
    # set state one in states
    states[0] = state_one
    states[-1] = state_L
    # generate the rest of the probabilites
    for i in range(1, L - 1): # fill the matrix with [0, 0]
        states[i] = [[0, 0] for i in range(L)]
    a, b, c = 0, 1, 2
    for row in states[1:-1]: # only generate probs between state one and state L
        row[a] = probs[0]
        row[b] = probs[1]
        row[c] = probs[2]
        a += 1
        b += 1
        c += 1
    return states

In [4]:
def rewards(n):
    reward = [0 for i in range(n)]
    reward[0] = 0.05
    reward[-1] = 1
    return reward

In [5]:
def delta(gamma, epsilon = 0.001):
    x = (epsilon*(1-gamma))/(2*gamma)
    return x


Value Iteration

In [6]:
def IV(state,gamma):
    reward = rewards(state)
    probs = prob_gen(state)
    V = [0 for i in range(state)]
    state_map = [i for i in range(state)]
    d = delta(gamma)
    pi = [-1 for i in range(state)]
    for i in range(40000):
        V_new = [0 for i in range(state)]
        max_diff = 0
        for s in state_map:
            max_val = 0
            for a in action:
                #compute state value
                val = reward[s]
                for s_next in state_map:
                    val += probs[s][s_next][a] * (gamma * V[s_next])
                #updating values
                max_val = max(max_val, val)
                #policy decision 
                if V[s] <= val:
                    pi[s] = action[a]
            
            V_new[s] = max_val

            max_diff = max(max_diff, abs(V[s] - V_new[s]))
        #update value function
        V = V_new
        #break when converging
        if max_diff < d:
            return V, pi, i
        

Policy Iteration

In [7]:
def PI(state,gamma):
    reward = rewards(state)
    probs = prob_gen(state)
    V = [0 for i in range(state)]
    state_map = [i for i in range(state)]
    d = delta(gamma)
    pi = [0 for i in range(state)]
    for i in range(10000):
        opt_pi_found = True
        #step 1) Policy Evaluation
        for j in range(10000):
            max_diff = 0
            for s in state_map:
                val = reward[s]
                for s_next in state_map:
                    val += probs[s][s_next][pi[s]] * gamma * V[s_next]
                #update max diff
                max_diff = max(max_diff, abs(val - V[s]))
                #update value function
                V[s] = val
            #break when converging
            if max_diff < d:
                break
        #Policy iteration
        for s in state_map:
            val_max = V[s]
            for a in action:
                val = reward[s]
                for s_next in state_map:
                    val += probs[s][s_next][a] * (gamma * V[s_next])

                if val > val_max and pi[s]!= a:
                    pi[s] = a
                    val_max = val
                    opt_pi_found = False
        if opt_pi_found:
            return V, pi, i

Point iii

In [41]:
p_three = [0.8, 0.85, 0.9, 0.92, 0.95, 0.97, 0.98, 0.99, 0.995]
head = ['gamma','value function', 'policy', '# of iterations']
iv_data = []
pi_data = []
for gamma in p_three:
    iv_value, iv_policy, iv_iterations = IV(6,gamma)
    pi_value, pi_policy, pi_iterations = PI(6,gamma)
    iv_data.append([gamma,iv_value,iv_policy,iv_iterations])
    pi_data.append([gamma,pi_value,pi_policy,pi_iterations])

#round value function
for results in iv_data:
    results[1] = list(np.around(np.array(results[1]),4))

for results in pi_data:
    results[1] = list(np.around(np.array(results[1]),4))

In [39]:
with open('./iv.csv','w') as file:
    writer = csv.writer(file)
    writer.writerow(head)
    for row in iv_data:
        writer.writerow(row)

with open('./pi.csv','w') as file:
    writer = csv.writer(file)
    writer.writerow(head)
    for row in pi_data:
        writer.writerow(row)

point iv

In [48]:
p_four = [6,8,10,14,18,25,40]
head = ['# of states','value function', 'policy', '# of iterations']
iv_data4 = []
pi_data4 = []
for L in p_four:
    iv_value, iv_policy, iv_iterations = IV(L,0.98)
    pi_value, pi_policy, pi_iterations = PI(L,0.98)
    iv_data4.append([L,iv_value,iv_policy,iv_iterations])
    pi_data4.append([L,pi_value,pi_policy,pi_iterations])

#round value function
for results in iv_data4:
    results[1] = list(np.around(np.array(results[1]),4))

for results in pi_data4:
    results[1] = list(np.around(np.array(results[1]),4))

In [49]:
with open('./iv4.csv','w') as file:
    writer = csv.writer(file)
    writer.writerow(head)
    for row in iv_data4:
        writer.writerow(row)

with open('./pi4.csv','w') as file:
    writer = csv.writer(file)
    writer.writerow(head)
    for row in pi_data4:
        writer.writerow(row)